In [3]:
from Crypto.Cipher import AES
from Crypto.Random import random
from hashlib import sha256

ca_key = (123911229913422142875264410911959753429255609781060012540592865384481400430784339515602026823919372067245349130530370079412035001372557207523878469898937327849288805682100737656179353201361997603827751534685210816357362467822736133251729781421390068496293277151621728336136046748401714936280146300069794792289, 65537)

P = 90217665064898209874945440415607645619739552209209829698859302616784804764535733761793578496057231124307702060703777047113648993244620436073319613409562635176198907683514151487667596066485528729421061091099275256268502121631878101098253283229895343146346757918637948217208979264467987857303672675468072062419

G = 13

def pad(msg):
    n = 16 - len(msg) % 16
    return msg + bytes([n]) * n

def unpad(msg):
    assert len(msg) > 0 and len(msg) % 16 == 0
    n = msg[-1]
    assert 1 <= n <= 16
    assert msg[-n:] == bytes([n]) * n
    return msg[:-n]

In [4]:
fromAliceAndBob1 = [
    "5eebbcd0e5ae7b4ef2469c371ada01d9ddf36f727350168b2474e69f4052cb9586b6e7c25c4da771fa3fb0db0d699ff241227099db8a500e2374c13dd37cfae7334469fb30f454231a388341d8251d8870e36bfcf25d6cc46853c10c7fa4015dfa6b07409ba97bd90eee4c01f2636e286476cd26f58a1c79036029942e6fc529",

"1929cf884ccfb53f778c2195991e0aeaf0e82f073e3ad4082611324ed94c24dc39e052ddf182446820a5a072f819453124e1dce5df601c8624e50fbde7b1958b7134e1c65dab0d25098c722ed9c03942498229c729da171c9716343608c59b244bb89b604aa3c9e13f6bd9b8bc613ff72c3ae0c99707674fcbb51059a8e36a16"
]

# myKey = random.getrandbits(1024)
myKey = 0
myKey -= myKey & 1

keys = [pow(int(from1, 16), myKey, P) for from1 in fromAliceAndBob1]
aesKeys = [int.to_bytes(key % (2**128), 16, "big") for key in keys]

toAliceAndBob1 = [hex(pow(G, myKey, P))[2: ] for _ in fromAliceAndBob1]

print("toAliceAndBob1 == {}".format(toAliceAndBob1))

toAliceAndBob1 == ['1', '1']


In [5]:
fromAliceAndBob2 = [
    "3645ea575c78f74a855fb4660c8602ac39193da7f8dd2157a9c41295f28e73d45d31abab8196e1eb8f547c5ef396a89a5a3c94f615a9cd1b83e9a158bbf06e5ed155e12929b4f78eb72466efe803419ade7a4220511bd1124fee3af23ac2d95092be5c38beb80bc27cdb99b402eff7a0baead7aae9af6e185d6ed00948cd7f51ef0721b7c7962d66d07e8ffc7caf16e7acd4737b340d0b194737c8aa7b18118f78952489c01f0df1e9df94bf619bc45bba094acfa6b8ba18ab6ec7822687c1099aedae46fa8b078a66b6409aeedb3ce1fd95f904aef9b0a312ba735eb3c28e01c2d8fa99577028ca39114699a214099256e00b3dcebd06f07194f4bc5bd894e0d06b367e97b09238db59f3f0b7d39b659d38c834861b811df751de0c969c27d0",
    "2725fcd336a4d44eb8409833f64bcc858c606a86deb78dd84199ae1e13d46c82f2301779e1f3ed4e152d11855f839b0ba1c5da51bf10ea4f8d68c01c3d21fca589f4a40fb83485f6dca1d269dc25b7b16d455aeaba7a1f4d4cda43d9f0d699feb206702b1fe46d74b44945d351a7c8c2e7d2b38ab919c6895a1dea017fe104842eb3f21ce9346b5324be27f290658c3f4af172b75871428458046b681e72a1334cdf993f54fc539a7e58acc69170166c8af324640532f88becd3c8296070e48f705201d8bc95b3e54d74ae633fffe9bc1d5cc93c0ed1dbbe2398878d06928be072798d1bb245ddb720cff7e51433ca257d5a84de52e304d4bb16925e4d937444aafe2461f379e9792eb3e901a0019bcac2e4ba289103e5bf23c9c4318a8b5491"
]

aesIvsAndCiphers = [bytes.fromhex(from2) for from2 in fromAliceAndBob2]
aesIvs = [aesIvAndCipher[: 16] for aesIvAndCipher in aesIvsAndCiphers]
ciphers = [aesIvAndCipher[16: ] for aesIvAndCipher in aesIvsAndCiphers]
aeses = [AES.new(aesKey, AES.MODE_CBC, aesIv) for aesKey, aesIv in zip(aesKeys, aesIvs)]
messages = [unpad(aes.decrypt(cipher)).decode() for aes, cipher in zip(aeses, ciphers)]
certs = messages

print("certs == {}".format(certs))

certs == ['416c696365_7d2e831fc7c37b4932604c66de52a3c7f631eee0badffb42f40899fb88ff19e41518fc8326f620aaeb54cecd8bd22a5fa8576848c131370c0fcb8fb0e093fbd0d76b73c47c04ffe2b913472595c9987d53b8cdffa4ff16f6a6176f215e6ded63e65089d763691a4cef91284e6934b95580e64854304b99e87215b582e201222d', '426f62_2453e551e17519b6dcfdae46f75b9d92dd943fcb5d2c71911adb83f43753dfba86be013dfb8c2413cbc70eac852e312040d3fbf471b57da2ae7a7aa95d39f3b972e34214f5cc69093d9929b2c5fc24684b0d5443dcb8338d1f52bb34f9913bd1e98c29302bbbd3cb532001adde7f06d93badcdee7b12b1b550a816537e950bad']
